## Import Libraries Packages, Downloads

In [797]:
import numpy as np
import pandas as pd

## Loading and Processing Data

In [799]:
Data = pd.read_csv('formatted_data.csv')

/var/folders/rs/6g2mhpzn6lq_k8wtscfvn5h80000gn/T/ipykernel_17374/3835250157.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  Data = pd.read_csv('formatted_data.csv')


In [800]:
def process(df): 

    #copy dataframe
    data = df.copy()

    #making a dictionary since we need to do a lot of conversion
    to_binary = {'Yes':1, 'No':0}

    #dropping columns we won't be able to use
    data = data.drop(['Busbreakdown_ID', 'Created_On', 
            'Informed_On','Incident_Number', 
            'Last_Updated_On', 'School_Age_or_PreK', 
            'School_Year', 'Route_Number', 'Schools_Serviced',
            'Run_Type', 'Bus_No', 'Bus_Company_Name'], axis = 1)

    #mapping literals to binary values
    data['Breakdown_or_Running_Late'] = data['Breakdown_or_Running_Late'].map({'Breakdown':1, 'Running Late':0})
    data['Has_Contractor_Notified_Parents'] = data['Has_Contractor_Notified_Parents'].map(to_binary)
    data['Has_Contractor_Notified_Schools'] = data['Has_Contractor_Notified_Schools'].map(to_binary)
    data['Have_You_Alerted_OPT'] = data['Have_You_Alerted_OPT'].map(to_binary)

    #getting mdate values
    data['Occurred_On'] = pd.to_datetime(data['Occurred_On'])
    data['Year'] = data['Occurred_On'].dt.year
    data['Month'] = data['Occurred_On'].dt.month
    data['Day'] = data['Occurred_On'].dt.day
    data['Hour'] = data['Occurred_On'].dt.hour
    data['Mins'] = data['Occurred_On'].dt.minute

    data = data.drop(['Occurred_On'], axis = 1)



    #one hot encode boro
    for boro in data['Boro'].unique():
        data[boro] = (data['Boro'] == boro).astype(int)

    # one hot encode breakdown reason
    for reason in data['Reason'].unique():
        data[reason] = (data['Reason'] == reason).astype(int)

    data = data.drop(['Boro'], axis = 1)
    data = data.drop(['Reason'], axis = 1)

    return data




In [801]:
df = process(Data)

In [802]:
x = df[df['How_Long_Delayed'].isin(['-1'])]
x['Breakdown_or_Running_Late'].sum()
df.isna().sum()


How_Long_Delayed                   0
Number_Of_Students_On_The_Bus      0
Has_Contractor_Notified_Schools    0
Has_Contractor_Notified_Parents    0
Have_You_Alerted_OPT               0
Breakdown_or_Running_Late          0
Year                               0
Month                              0
Day                                0
Hour                               0
Mins                               0
New Jersey                         0
Manhattan                          0
Bronx                              0
Westchester                        0
Brooklyn                           0
Rockland County                    0
Nassau County                      0
NaN                                0
Queens                             0
Staten Island                      0
All Boroughs                       0
Connecticut                        0
Heavy Traffic                      0
Flat Tire                          0
Other                              0
Won`t Start                        0
M

In [803]:
df.head()

,How_Long_Delayed,Number_Of_Students_On_The_Bus,Has_Contractor_Notified_Schools,Has_Contractor_Notified_Parents,Have_You_Alerted_OPT,Breakdown_or_Running_Late,Year,Month,Day,Hour,...,Heavy Traffic,Flat Tire,Other,Won`t Start,Mechanical Problem,Problem Run,Accident,Late return from Field Trip,Delayed by School,Weather Conditions
0,-1,11,1,0,1,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
1,0-30,2,1,1,0,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
2,0-30,8,1,1,1,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
3,0-30,6,0,0,0,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
4,-1,6,0,0,0,0,2015,11,5,8,...,1,0,0,0,0,0,0,0,0,0
